<a href="https://colab.research.google.com/github/cbedart/EGBSL_IAEA/blob/main/EGBSL_IAEA_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<center><h1>EGBSL - IAES - Part #3</h1></center>**

---

In [ ]:
# @title Loading of prerequisites packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns; sns.set()
from sklearn.decomposition import PCA, KernelPCA
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import Image, display, clear_output, Javascript

import plotly.express as px

from sklearn import tree
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut

from sklearn.preprocessing import StandardScaler

import seaborn as sns; sns.set()
import plotly.express as px
import pydotplus

################################################################################

launch_rf_hyperparameters = 0

################################################################################

def tree_graph_to_png(tree, feature_names, class_names, png_file_to_save):
    tree_str = export_graphviz(tree, feature_names=feature_names, class_names=class_names,
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)
    graph.write_png(png_file_to_save)




np.seterr(divide='ignore', invalid='ignore')

!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/computed_desc_GR_tosave.pkl
!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/desc_GR_clustered_normalized_noNA.pkl
!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/desc_GR_PCA_53c.pkl

computed_desc_GR_tosave = pd.read_pickle("computed_desc_GR_tosave.pkl")
desc_GR_clustered_prep = pd.read_pickle("desc_GR_clustered_normalized_noNA.pkl")
desc_GR_PCA_53c = pd.read_pickle("desc_GR_PCA_53c.pkl")

print("\n\n\033[1mPrerequisites succesfully installed !\033[0m")

In [ ]:
#@title Prepare the data for the neural network models
desc_GR_DTs = desc_GR_clustered_prep

pic50_limit = 8

ygroup2 = []
ycat2 = []
for i in desc_GR_DTs.iloc[:,2]:
  if i > pic50_limit:
    ygroup2.append("Active")
    ycat2.append(1)
  else:
    ygroup2.append("Less Active")
    ycat2.append(0)

desc_GR_activity2 = pd.concat([desc_GR_DTs.iloc[:,:4], pd.DataFrame(ygroup2, columns=["Activity_Label"]), pd.DataFrame(ycat2, columns=["Activity"]), desc_GR_DTs.iloc[:,4:]], axis = "columns")

X2 = desc_GR_activity2.iloc[:,6:]
y2 = desc_GR_activity2.loc[:,"Activity"]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

################################################################################

print("\nWe setup the pIC50 limit to {}, with :".format(pic50_limit))
print("- A total of {} active compounds (pIC50 < {}) - Coded as 1".format(desc_GR_activity2["Activity"].value_counts()[1], pic50_limit))
print("- A total of {} less active compounds (pIC50 > {}) - Coded as 0".format(desc_GR_activity2["Activity"].value_counts()[0], pic50_limit))

print("\n\033[1mDone !\033[0m \n\n")

In [ ]:
# @title Build your own artificial neural network
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 700})'''))

# @markdown Number of hidden layers:
nb_layers = 3 # @param [1,2,3,4,5] {type:"raw"}

# @markdown Number of nodes per hidden layers:
layer1 = 100 # @param {type:"integer"}
layer2 = 100 # @param {type:"integer"}
layer3 = 100 # @param {type:"integer"}
layer4 = 100 # @param {type:"integer"}
layer5 = 100 # @param {type:"integer"}

# @markdown Activation function:
activation_function = "logistic" # @param ["identity", "logistic", "tanh", "relu"] {type:"string"}

solver_opt = "adam"

################################################################################

if nb_layers == 1:
  if layer1 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1)
elif nb_layers == 2:
  if layer1 <= 0 or layer2 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2)
elif nb_layers == 3:
  if layer1 <= 0 or layer2 <= 0 or layer3 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2, layer3)
elif nb_layers == 4:
  if layer1 <= 0 or layer2 <= 0 or layer3 <= 0 or layer4 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2, layer3, layer4)
else :
  if layer1 <= 0 or layer2 <= 0 or layer3 <= 0 or layer4 <= 0 or layer5 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2, layer3, layer4, layer5)

nn1 = MLPClassifier(hidden_layer_sizes=custom_layer_sizes, activation = activation_function, verbose = True, solver = solver_opt, alpha=1e-4, max_iter= 2000)
# nn1 = MLPClassifier(hidden_layer_sizes=custom_layer_sizes, activation = 'identity', verbose = True, solver = 'lbfgs', alpha=1e-5, max_iter= 2000)
nn1 = nn1.fit(X_train2, y_train2)
y_pred_nn1 = nn1.predict(X_test2)

################################################################################

print("\n\n\033[1mQuality assessment metrics - {} hidden layers / {} architecture:\033[0m\n".format(nb_layers, custom_layer_sizes))
print("Accuracy on the train set = {}\n".format(round(nn1.score(X_train2, y_train2),3)))
print("Accuracy on the test set = {}\n".format(round(nn1.score(X_test2, y_test2),3)))

print("Confusion matrix:")
fig, axn = plt.subplots(1,2, figsize=(12,3))
axn[0].grid(False) ; axn[1].grid(False)
confusion1_nn1 = ConfusionMatrixDisplay.from_estimator(nn1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[0], cmap=plt.cm.Blues, normalize=None)
confusion1_nn1.ax_.set_title("Confusion matrix, without normalization")
# confusion1_nn1
confusion2_nn1 = ConfusionMatrixDisplay.from_estimator(nn1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[1], cmap=plt.cm.Blues, normalize="true")
confusion2_nn1.ax_.set_title("Normalized confusion matrix")
# confusion2_nn1
plt.show()

print("\nPrecision = {}".format(round(confusion1_nn1.confusion_matrix[1,1] / (confusion1_nn1.confusion_matrix[1,1] + confusion1_nn1.confusion_matrix[0,1]), 3)))
print("\nRecall = {}".format(round(confusion1_nn1.confusion_matrix[1,1] / (confusion1_nn1.confusion_matrix[1,1] + confusion1_nn1.confusion_matrix[1,0]), 3)))

nn1_acc_pr_rec = [round(nn1.score(X_train2, y_train2),3),
                  round(nn1.score(X_test2, y_test2),3),
                  round(confusion1_nn1.confusion_matrix[1,1] / (confusion1_nn1.confusion_matrix[1,1] + confusion1_nn1.confusion_matrix[0,1]), 3),
                  round(confusion1_nn1.confusion_matrix[1,1] / (confusion1_nn1.confusion_matrix[1,1] + confusion1_nn1.confusion_matrix[1,0]), 3)]